# Multinomial classification

## Softmax Classification

#### tf.nn.softmax() : exp(logits) / reduce_sum(exp(logits), dim) <br>
#### tf.log() : log function <br>
#### tf.arg_max() : 가장 확률이 높은 값을 출력

In [7]:
import tensorflow as tf

x_data = [[1,2,1,1], [2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]]

X = tf.placeholder("float",[None, 4])
Y = tf.placeholder("float",[None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4,nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

# tf.nn.softmax computes sotfmax activations
# sotfmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y: y_data}))
    
    all = sess.run(hypothesis,feed_dict={X:[[1,11,7,9], [1,3,4,3], [1,1,0,1]]})
    print(all,"\n", sess.run(tf.arg_max(all,1)))    

0 9.937427
200 0.6925634
400 0.59194463
600 0.5039958
800 0.41726428
1000 0.32955775
1200 0.24681036
1400 0.21834144
1600 0.19961514
1800 0.18371303
2000 0.17005676
[[7.92053118e-02 9.20782566e-01 1.21806515e-05]
 [5.36878765e-01 3.93875867e-01 6.92454204e-02]
 [2.65897988e-08 4.46837017e-04 9.99553144e-01]] 
 [1 0 2]


## Fancy Softmax Classifier

#### tf.one_hot() : 가장 큰 확률을 가진 값을 one hot shape으로 return <br>
#### tf.reshape() :  shape을 바꿔줌<br>
#### tf.nn.softmax_cross_entropy_with_logits() : tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis = 1))<br>

In [4]:
import tensorflow as tf
import numpy as np

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None,1])

# Convert y_data to one_hot
Y_one_hot = tf.one_hot(Y,nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1,nb_classes])

W = tf.Variable(tf.random_normal([16, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

# tf.nn/softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W)+b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

#### tf.argmax() : max 값 return <br>



In [9]:
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Lauch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict = {X: x_data, Y:y_data})
        if step % 200 == 0:
            loss,acc = sess.run([cost, accuracy], feed_dict = {X: x_data, Y: y_data})
            print("Step : {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X:x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p,y in zip(pred, y_data.flatten()):
        print("[{}] Prediction : {} True Y: {}".format(p==int(y), p, int(y)))

Step :     0	Loss: 3.694	Acc: 39.60%
Step :   200	Loss: 0.424	Acc: 89.11%
Step :   400	Loss: 0.261	Acc: 95.05%
Step :   600	Loss: 0.192	Acc: 96.04%
Step :   800	Loss: 0.153	Acc: 96.04%
Step :  1000	Loss: 0.126	Acc: 99.01%
Step :  1200	Loss: 0.107	Acc: 99.01%
Step :  1400	Loss: 0.092	Acc: 99.01%
Step :  1600	Loss: 0.081	Acc: 99.01%
Step :  1800	Loss: 0.072	Acc: 100.00%
[True] Prediction : 0 True Y: 0
[True] Prediction : 0 True Y: 0
[True] Prediction : 3 True Y: 3
[True] Prediction : 0 True Y: 0
[True] Prediction : 0 True Y: 0
[True] Prediction : 0 True Y: 0
[True] Prediction : 0 True Y: 0
[True] Prediction : 3 True Y: 3
[True] Prediction : 3 True Y: 3
[True] Prediction : 0 True Y: 0
[True] Prediction : 0 True Y: 0
[True] Prediction : 1 True Y: 1
[True] Prediction : 3 True Y: 3
[True] Prediction : 6 True Y: 6
[True] Prediction : 6 True Y: 6
[True] Prediction : 6 True Y: 6
[True] Prediction : 1 True Y: 1
[True] Prediction : 0 True Y: 0
[True] Prediction : 3 True Y: 3
[True] Prediction : 0